In [2]:
import pandas as pd

In [3]:
inputd = pd.read_csv('SampleInput.csv')
inputd.head()

,LocalTime,Symbol,EventType,Side,FillSize,FillPrice,FillExchange
0,9:30:00.000,AES,TRADE,t,100,11.14,NYSE
1,9:30:01.000,AES,TRADE,b,100,11.16,NASDAQ
2,9:30:02.000,CX,TRADE,b,100,7.50,NASDAQ
3,9:30:03.000,CX,TRADE,t,100,7.51,NASDAQ
4,9:30:04.000,CX,TRADE,b,200,7.50,NASDAQ


* SymbolBought
    * Number of shares of the stock bought
* SymbolSold
    * Number of shares of the stock sold
* SymbolPosition
    * Number of shares of a stock the portfolio currently holds or owes
    * Dependent on the number of shares we bought vs. sold, the position can be:
        * Long (positive number) if we bought more shares than we sold
        * Short (negative number) if we sold more than we bought
        * Flat (zero) if we do not have a position in the given stock
* SymbolNotional
    * Value of the shares bought or sold, == FillSize * FilledPrice
* ExchangeBought
    * Number of shares bought on the current exchange, across all symbols
* ExchangeSold
    * Number of shares sold on the current exchange, across all symbols
* TotalBought
    * Total number of shares bought across all symbols
* TotalSold
    * Total number of shares sold across all symbols
* TotalBoughtNotional
    * Total value (SymbolNotional) of all shares bought across all symbols
* TotalSoldNotional
    * Total value (SymbolNotional) of all shares sold across all symbols


In [4]:
outputd = pd.read_csv('SampleOutput.csv')
outputd.head()

,LocalTime,Symbol,EventType,Side,FillSize,FillPrice,FillExchange,SymbolBought,SymbolSold,SymbolPosition,SymbolNotional,ExchangeBought,ExchangeSold,TotalBought,TotalSold,TotalBoughtNotional,TotalSoldNotional
0,9:30:00.000,AES,TRADE,t,100,11.14,NYSE,0,100,-100,1114.0,0,100,0,100,0.0,1114.0
1,9:30:01.000,AES,TRADE,b,100,11.16,NASDAQ,100,100,0,1116.0,100,0,100,100,1116.0,1114.0
2,9:30:02.000,CX,TRADE,b,100,7.50,NASDAQ,100,0,100,750.0,200,0,200,100,1866.0,1114.0
3,9:30:03.000,CX,TRADE,t,100,7.51,NASDAQ,100,100,0,751.0,200,100,200,200,1866.0,1865.0
4,9:30:04.000,CX,TRADE,b,200,7.50,NASDAQ,300,100,200,1500.0,400,100,400,200,3366.0,1865.0


In [14]:
def csv_reader(file_name):
    lines = (line for line in open(file_name))
    list_line = (s.rstrip().split(",") for s in lines) 
    cols = next(list_line)
    for row in list_line:
        yield row
next(csv_reader('SampleInput.csv'))

['9:30:00.000', 'AES', 'TRADE', 't', '100', '11.14', 'NYSE']

In [16]:
count = 0
for row in csv_reader('SampleInput.csv'):
    count +=1
count

12

In [17]:
for row in csv_reader('SampleInput.csv'):
    extended = (row +
                [0 if row[3] != 'b' else row[4]] + 
                [0 if row[3] != 'b' else row[4]])
    print(extended)

['9:30:00.000', 'AES', 'TRADE', 't', '100', '11.14', 'NYSE', 0, 0]
['9:30:01.000', 'AES', 'TRADE', 'b', '100', '11.16', 'NASDAQ', '100', '100']
['9:30:02.000', 'CX', 'TRADE', 'b', '100', '7.50', 'NASDAQ', '100', '100']
['9:30:03.000', 'CX', 'TRADE', 't', '100', '7.51', 'NASDAQ', 0, 0]
['9:30:04.000', 'CX', 'TRADE', 'b', '200', '7.50', 'NASDAQ', '200', '200']
['9:30:05.000', 'CX', 'TRADE', 't', '100', '7.50', 'NASDAQ', 0, 0]
['9:30:06.000', 'CX', 'TRADE', 't', '200', '7.48', 'NASDAQ', 0, 0]
['9:30:07.000', 'AIG', 'TRADE', 't', '100', '56.40', 'NASDAQ', 0, 0]
['9:30:08.000', 'AIG', 'TRADE', 't', '100', '56.39', 'NYSE', 0, 0]
['9:30:09.000', 'AIG', 'TRADE', 'b', '100', '56.38', 'NASDAQ', '100', '100']
['9:30:10.000', 'AIG', 'TRADE', 'b', '100', '56.38', 'NASDAQ', '100', '100']
['9:30:11.000', 'AIG', 'TRADE', 't', '100', '56.72', 'NASDAQ', 0, 0]


In [38]:
def logic(row: list, groud_truth: dict)-> dict:
    tail = [0] * 10
    row[4] = int(row[4])
    row[5] = float(row[5])
    tail[3] = row[4] * row[5]
    if row[3] == 'b':
        if row[1] not in groud_truth.keys():
            groud_truth[row[1]] = row[4]
            tail[0] = row[4]
        else:
            groud_truth[row[1]] += row[4]
            tail[0] += row[4]
        if row[6] not in groud_truth.keys():
            groud_truth[row[6]] = {'b': row[4], 's': 0}
        else:
            groud_truth[row[6]]['b'] += row[4]            
    else: 
        if row[1] not in groud_truth.keys():
            groud_truth[row[1]] = -row[4]
            tail[1] = row[4]
        else:
            groud_truth[row[1]] -= row[4]
            tail[1] += row[4]
        if row[6] not in groud_truth.keys():
            groud_truth[row[6]] = {'b': 0, 's': row[4]}
        else:
            groud_truth[row[6]]['s'] += row[4]  
    tail[2] = groud_truth[row[1]]
    tail[4] = groud_truth[row[6]]['b']
    tail[5] = groud_truth[row[6]]['s']
    return tail, groud_truth

In [39]:
the_truth = {}
for row in csv_reader('SampleInput.csv'):
    extended, the_truth = logic(row, the_truth)
    print(extended)

[0, 100, -100, 1114.0, 0, 100, 0, 0, 0, 0]
[100, 0, 0, 1116.0, 100, 0, 0, 0, 0, 0]
[100, 0, 100, 750.0, 200, 0, 0, 0, 0, 0]
[0, 100, 0, 751.0, 200, 100, 0, 0, 0, 0]
[200, 0, 200, 1500.0, 400, 100, 0, 0, 0, 0]
[0, 100, 100, 750.0, 400, 200, 0, 0, 0, 0]
[0, 200, -100, 1496.0, 400, 400, 0, 0, 0, 0]
[0, 100, -100, 5640.0, 400, 500, 0, 0, 0, 0]
[0, 100, -200, 5639.0, 0, 200, 0, 0, 0, 0]
[100, 0, -100, 5638.0, 500, 500, 0, 0, 0, 0]
[100, 0, 0, 5638.0, 600, 500, 0, 0, 0, 0]
[0, 100, -100, 5672.0, 600, 600, 0, 0, 0, 0]


In [28]:
outputd.head()

,LocalTime,Symbol,EventType,Side,FillSize,FillPrice,FillExchange,SymbolBought,SymbolSold,SymbolPosition,SymbolNotional,ExchangeBought,ExchangeSold,TotalBought,TotalSold,TotalBoughtNotional,TotalSoldNotional
0,9:30:00.000,AES,TRADE,t,100,11.14,NYSE,0,100,-100,1114.0,0,100,0,100,0.0,1114.0
1,9:30:01.000,AES,TRADE,b,100,11.16,NASDAQ,100,100,0,1116.0,100,0,100,100,1116.0,1114.0
2,9:30:02.000,CX,TRADE,b,100,7.50,NASDAQ,100,0,100,750.0,200,0,200,100,1866.0,1114.0
3,9:30:03.000,CX,TRADE,t,100,7.51,NASDAQ,100,100,0,751.0,200,100,200,200,1866.0,1865.0
4,9:30:04.000,CX,TRADE,b,200,7.50,NASDAQ,300,100,200,1500.0,400,100,400,200,3366.0,1865.0


In [40]:
the_truth

{'AES': 0,
 'NYSE': {'b': 0, 's': 200},
 'NASDAQ': {'b': 600, 's': 600},
 'CX': -100,
 'AIG': -100}